In [52]:
import requests
from bs4 import BeautifulSoup
import time 
from tqdm.notebook import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By

In [53]:
browser = webdriver.Firefox()

In [65]:
def find_drinks_on_page(browser, page_url, page_number): 
    drink_urls = [] 

    browser.get(f'{page_url}&page={page_number}')
    browser.implicitly_wait(7) 
    browser.execute_script("window.scrollBy(0, document.body.scrollHeight);")  # Scroll down by 400 pixels
    browser.implicitly_wait(3)  # wait for 3 seconds
    time.sleep(3)
    browser.execute_script("window.scrollBy(0, document.body.scrollHeight);")  # Scroll down by 400 pixels
    
    for element in browser.find_elements(By.TAG_NAME, 'ul'): 
        if element.get_attribute('class') != 'ProductListPage CategoryProductList-Page CategoryProductList-Page_layout_grid': 
            continue

        for drink in element.find_elements(By.TAG_NAME, 'a'): 
            drink_urls.append(drink.get_attribute('href') )

    return drink_urls


In [69]:
urls = find_drinks_on_page(browser, 'https://www.monin.com/us/recipes/drinks?customFilters=category_id:14', 7)

urls


['https://www.monin.com/us/golden-cold-brew',
 'https://www.monin.com/us/golden-creme-cold-brew',
 'https://www.monin.com/us/golden-latte-steamer',
 'https://www.monin.com/us/golden-lion-latte',
 'https://www.monin.com/us/golden-maple-iced-latte',
 'https://www.monin.com/us/grandmas-blueberry-delight-latte',
 'https://www.monin.com/us/hansel-gretel-latte',
 'https://www.monin.com/us/happy-harvest-apple-cobbler',
 'https://www.monin.com/us/harvest-nut-latte',
 'https://www.monin.com/us/hazelnut-cream-mocha',
 'https://www.monin.com/us/hazelnut-iced-coffee',
 'https://www.monin.com/us/hazelnut-soy-latte',
 'https://www.monin.com/us/holiday-cold-brew',
 'https://www.monin.com/us/holiday-joy-mocha',
 'https://www.monin.com/us/honey-almond-latte',
 'https://www.monin.com/us/honey-bee-cortado',
 'https://www.monin.com/us/honey-cinnamon-latte',
 'https://www.monin.com/us/honey-latte',
 'https://www.monin.com/us/honey-maple-latte',
 'https://www.monin.com/us/honey-roasted-pecan-latte',
 'https

In [4]:
pages = {
    'lemonade': [
        ('https://www.monin.com/us/recipes/drinks?customFilters=category_id:18', 9)
    ], 

    'coffee':  [
        ('https://www.monin.com/us/recipes/drinks?customFilters=category_id:14', 16)
    ],

    # 'milkshake': [
    #     ('', 1)
    # ],

    # 'hot_tea': [
    #     ('', 1)
    # ],

    'cold_tea': [
        ('https://www.monin.com/us/recipes/drinks?customFilters=category_id:33', 5)
    ],

    'alcohol': [
        ('https://www.monin.com/us/recipes/drinks?customFilters=category_id:16', 32)
    ],

    'smoothie': [
        ('https://www.monin.com/us/recipes/drinks?customFilters=category_id:558', 3), 
        ('https://www.monin.com/us/recipes/drinks?customFilters=category_id:724', 2), 
    ],      

    'punch': [
        ('https://www.monin.com/us/recipes/drinks?customFilters=category_id:163', 2)
    ],    
}

In [19]:
for category in pages: 
        for category_url, pages_number in pages[category]: 
                for page in range(1, pages_number+1): 
                        page_url = category_url + f"&page={page}"
                        page_request = requests.get(page_url, headers={"Content-Type":"text", 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'})
                        page_content = page_request.content
                        print(page_content)
                        break

                        soup = BeautifulSoup(page_content, 'html.parser')
                        drinks_list = soup.find_all('ul')
                        print(len(drinks_list))
                        print(page_url, page_request.status_code)
        break

b'<!DOCTYPE html>\n<html lang="en" dir="ltr" xmlns="http://www.w3.org/1999/xhtml">\n<head>\n<meta charset="utf-8" /><script type="text/javascript">(window.NREUM||(NREUM={})).init={privacy:{cookies_enabled:true},ajax:{deny_list:[]}};(window.NREUM||(NREUM={})).loader_config={xpid:"UwQHU1ZaGwAHV1BRAwQPVg==",licenseKey:"eb677859e7",applicationID:"373134580"};;/*! For license information please see nr-loader-spa-1.265.0.min.js.LICENSE.txt */\n(()=>{var e,t,r={8122:(e,t,r)=>{"use strict";r.d(t,{a:()=>i});var n=r(944);function i(e,t){try{if(!e||"object"!=typeof e)return(0,n.R)(3);if(!t||"object"!=typeof t)return(0,n.R)(4);const r=Object.create(Object.getPrototypeOf(t),Object.getOwnPropertyDescriptors(t)),o=0===Object.keys(r).length?e:r;for(let a in o)if(void 0!==e[a])try{if(null===e[a]){r[a]=null;continue}Array.isArray(e[a])&&Array.isArray(t[a])?r[a]=Array.from(new Set([...e[a],...t[a]])):"object"==typeof e[a]&&"object"==typeof t[a]?r[a]=i(e[a],t[a]):r[a]=e[a]}catch(e){(0,n.R)(1,e)}return r}c